<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#План-действий:" data-toc-modified-id="План-действий:-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>План действий:</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

## План действий:
1. Ознакомиться с файлом, изучить его, подготовим данные
2. Исследуем баланс классов, обучим модель без учета дисбаланса
3. Исследовать качество моделей, изменив дисбаланс
4. Проверить качество модели на тестовой выборке
5. Сделать общий вывод

## Подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle


In [2]:

df.info()
print(df.head(10))
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB
   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio      

Таким образом видно, что мы имеем датафрейм, состоящий из 10000 строк и 14 признаков, из них 3 являются float64, 8 - int64 и 3- object. Также видно, что столбце Tenure имеет 909 пропущенных значений, которые необходимо заполнить. Также можно заметить, что 25% имеют 0 баланс, скорее всего клиенты банка просто вывели свои деньги из банка.

Приведем название столбцов к нормльному виду

In [3]:
df.columns = ['row_number', 'customer_id', 'surname', 'credit_score', 'geography', 'gender', 'age', 'tenure', 'balance', 'num_of_products', 'has_cr_card', 'is_active_member', 'estimated_salary', 'exited']
df

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


Удалим ненужные столбцы, которые являются информацией для знакомства с клиентом, но не влияют уйдет ли клиент из банка, а именно столбцы: row_number, surname, customer_id.

In [4]:
df = df.drop(['row_number', 'surname', 'customer_id'], axis=1)

Заменим пропущенные значения в столбце "Tenure" на медианное значение по этому столбцу. Возможно это новые пользователи или те, которые уже вышли, но так как некоторые уже покинули банк, то скорее всего это клиенты, которые уже несколько лет в банке, поэтому лучше заменить значение на медианное. Также среднее значение и медианное в данном столбце примерно равны. Также можно удалить данные значения, так как то, сколько лет провел клиент в банке не сильно повлияет на его уход, он может уйти в первый год, а может и через 5 лет.

In [5]:
df_first = df.fillna(value = 5)
print(df_first['tenure'].isna().sum())
print(df_first.shape)

0
(10000, 11)


Создали новый датафрейм, в котором нет пропущенных значений. Но также сделаем новый датасет, в котором будут удалены пропущенные значения и в дальнейшем проверим как изменится качество модели.

In [6]:
df_new = df.dropna().reset_index(drop = True)
print(df_new.shape)

(9091, 11)


Таким образом было создано два датасета, для того, чтобы рассмотреть две разные модели и увидеть, качество какой модели будет лучше.

Преобразуем категориальные признаки в численные для обоих датасетов.

In [7]:
df_first_ohe = pd.get_dummies(df_first, drop_first = True)
df_new_ohe = pd.get_dummies(df_new, drop_first = True)

**Таким образом была произведена подготовка данных к дальнейшему исследованию, были заменены категориальные данные на целочисленные, создано два датасета для их дальнейшего исследования: с заменой пропущенных значений и с удалением пропущенных значений, также были удалены ненудные столбцы**

## Исследование задачи

# Работа с первым датасетом с заменой значений

Разделим выборки на тренировочную (которая разделится на обучающую и валидационную) и тестовую выборки. Применим для этого метод sample. По правилам деления выборки должны разделиться на 60 - 20 -20 %, т.е. они должны равняться 6000 - 2000 - 2000. Для начала разделим выборку на тренировочную и тестовую.

In [8]:
train_first = df_first_ohe.sample(frac = 0.8, random_state = 123).copy()
print(train_first.shape)

test_first = df_first_ohe[~df_first_ohe.index.isin(train_first.index)].copy()
print(test_first.shape)

(8000, 12)
(2000, 12)


Таким образом данные были разделены на обучающую и тестовую выборки. Но для проверки качества модели нужна также валидационная выборка, получим её из тренировочной выборки, чтобы получить число 2000, нужно взять 25 процентов от размера train_first.

In [9]:
df_train_first, df_valid_first = train_test_split(train_first, test_size=0.25, random_state=123)
print(df_train_first.shape)
print(df_valid_first.shape)

(6000, 12)
(2000, 12)


Сохраним признаки и цели для каждой выборки в отдельных переменных features_train_first, target_train_first; features_valid_first, target_valid_first; features_test_first, target_test_first и проверим их размер, чтобы убедиться, что в признаках осталось 11 столбцов.

In [10]:
features_train_first = df_train_first.drop(['exited'], axis=1)
target_train_first = df_train_first['exited']

features_valid_first = df_valid_first.drop(['exited'], axis=1)
target_valid_first = df_valid_first['exited']

features_test_first = test_first.drop(['exited'], axis=1)
target_test_first = test_first['exited']

print(features_train_first.shape)
print(target_train_first.shape)

print(features_valid_first.shape)
print(target_valid_first.shape)

print(features_test_first.shape)
print(target_test_first.shape)

(6000, 11)
(6000,)
(2000, 11)
(2000,)
(2000, 11)
(2000,)


Таким образом были разделены данные на тренировочную, валидационную и тестовую выборки в соотношении 60:20:20. Далее приступим к стандартизации классов методом масштабирования признаков.

In [11]:
# Выделяем те признаки, которые необходимо масштабировать
numeric = ['credit_score', 'age', 'balance', 'estimated_salary']

scaler = StandardScaler()
scaler.fit(features_train_first[numeric])
features_train_first[numeric] =  scaler.transform(features_train_first[numeric])
features_valid_first[numeric] =  scaler.transform(features_valid_first[numeric])
features_test_first[numeric] =  scaler.transform(features_test_first[numeric])
features_train_first.head()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,geography_Germany,geography_Spain,gender_Male
2256,0.479630,-0.568592,2.0,1.393707,1,1,1,-1.609849,0,0,1
9202,-1.257624,-0.759496,9.0,0.715344,1,1,0,-1.638045,0,0,1
8544,0.407245,0.672285,7.0,-0.393582,2,1,1,-0.459033,0,0,0
8818,0.283155,-0.186784,3.0,-1.230201,2,1,1,-1.065188,0,0,0
8017,-1.971139,1.054093,5.0,0.529178,1,1,0,0.984366,1,0,1


Проверим сбалансированы ли данные по целевому признаку методом value_counts().

In [12]:
df_first_ohe['exited'].value_counts()

0    7963
1    2037
Name: exited, dtype: int64

Как можно заметить данные не сбалансированы, они примерно соотносятся как 4:1. Попробуем построить модель по данным с дисбалансом и проверим качество модели.

**Начнем проверку с Дерева решений**


In [13]:
model = DecisionTreeClassifier(random_state=123, max_depth = 3)
model.fit(features_train_first, target_train_first)
predicted_valid_first = model.predict(features_valid_first)

print('Accuracy:', accuracy_score(target_valid_first, predicted_valid_first))
print('F1:', f1_score(target_valid_first, predicted_valid_first))

#Сравним модель со случайной
probabilities_valid_first = model.predict_proba(features_valid_first)
probabilities_one_valid_first = probabilities_valid_first[:, 1]

print('ROC_AUC:', roc_auc_score(target_valid_first, probabilities_one_valid_first))

Accuracy: 0.8385
F1: 0.4262877442273535
ROC_AUC: 0.7817634680334605


**Случайный лес**

In [14]:
model = RandomForestClassifier(random_state=123)
model.fit(features_train_first, target_train_first)
predicted_valid_first = model.predict(features_valid_first)

print('Accuracy:', accuracy_score(target_valid_first, predicted_valid_first))
print('F1:', f1_score(target_valid_first, predicted_valid_first))

#Сравним модель со случайной
probabilities_valid_first = model.predict_proba(features_valid_first)
probabilities_one_valid_first = probabilities_valid_first[:, 1]

print('ROC_AUC:', roc_auc_score(target_valid_first, probabilities_one_valid_first))

Accuracy: 0.851
F1: 0.5130718954248367
ROC_AUC: 0.8072507208443973


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


**Логистическая регрессия**

In [15]:
model = LogisticRegression(random_state=123, solver='liblinear')
model.fit(features_train_first, target_train_first)
predicted_valid_first = model.predict(features_valid_first)

print('Accuracy:', accuracy_score(target_valid_first, predicted_valid_first))
print('F1:', f1_score(target_valid_first, predicted_valid_first))

#Сравним модель со случайной
probabilities_valid_first = model.predict_proba(features_valid_first)
probabilities_one_valid_first = probabilities_valid_first[:, 1]

print('ROC_AUC:', roc_auc_score(target_valid_first, probabilities_one_valid_first))

Accuracy: 0.8185
F1: 0.3059273422562141
ROC_AUC: 0.7752263871009626


Как можно заметить для всех трех моделей точность высока, также как и значение AUC_ROC - площадь под кривой, но данные модели плохо предсказывают целевой признак со значением класс 1, только случайный лес показывает значение превышающее 0.51, однако этого не хватает до 0.59, поэтому необходимо бороться с дисбалансом классов.

In [66]:
df_all = pd.DataFrame([[1,'DesicionTree', 0.8385, 0.4262877442273535, 0.7817634680334605],
                  [2,'RandomForest', 0.851, 0.5130718954248367, 0.8072507208443973],
                  [3,'LogisticRegression', 0.8185, 0.3059273422562141, 0.7752263871009626]], 
columns=['id','name_of_method', 'Accuracy', 'F1', 'ROC_AUC'])
df_all

,id,name_of_method,Accuracy,F1,ROC_AUC
0,1,DesicionTree,0.8385,0.426288,0.781763
1,2,RandomForest,0.8510,0.513072,0.807251
2,3,LogisticRegression,0.8185,0.305927,0.775226


# Работа со вторым датасетом с удаленными значениями

Разделим выборки на тренировочную (которая разделится на обучающую и валидационную) и тестовую выборки. Применим для этого метод sample. По правилам деления выборки должны разделиться на 60 - 20 -20 %, т.е. они должны равняться 6000 - 2000 - 2000. Для начала разделим выборку на тренировочную и тестовую.

In [16]:
train_new = df_new_ohe.sample(frac = 0.8, random_state = 123).copy()
print(train_new.shape)

test_new = df_new_ohe[~df_new_ohe.index.isin(train_new.index)].copy()
print(test_new.shape)

(7273, 12)
(1818, 12)


Таким образом данные были разделены на обучающую и тестовую выборки. Но для проверки качества модели нужна также валидационная выборка, получим её из тренировочной выборки, чтобы получить число 2000, нужно взять 25 процентов от размера train_new.

In [17]:
df_train_new, df_valid_new = train_test_split(train_new, test_size=0.25, random_state=123)
print(df_train_new.shape)
print(df_valid_new.shape)

(5454, 12)
(1819, 12)


Сохраним признаки и цели для каждой выборки в отдельных переменных features_train_new, target_train_new; features_valid_new, target_valid_new; features_test_new, target_test_new и проверим их размер, чтобы убедиться, что в признаках осталось 11 столбцов

In [18]:
features_train_new = df_train_new.drop(['exited'], axis=1)
target_train_new = df_train_new['exited']

features_valid_new = df_valid_new.drop(['exited'], axis=1)
target_valid_new = df_valid_new['exited']

features_test_new = test_new.drop(['exited'], axis=1)
target_test_new = test_new['exited']

print(features_train_new.shape)
print(target_train_new.shape)

print(features_valid_new.shape)
print(target_valid_new.shape)

print(features_test_new.shape)
print(target_test_new.shape)

(5454, 11)
(5454,)
(1819, 11)
(1819,)
(1818, 11)
(1818,)


Таким образом были разделены данные на тренировочную, валидационную и тестовую выборки в соотношении 60:20:20. Далее приступим к стандартизации классов методом масштабирования признаков.

In [19]:
# Выделяем те признаки, которые необходимо масштабировать
numeric = ['credit_score', 'age', 'balance', 'estimated_salary']

scaler = StandardScaler()
scaler.fit(features_train_new[numeric])
features_train_new[numeric] =  scaler.transform(features_train_new[numeric])
features_valid_new[numeric] =  scaler.transform(features_valid_new[numeric])
features_test_new[numeric] =  scaler.transform(features_test_new[numeric])
features_train_new.head()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,geography_Germany,geography_Spain,gender_Male
8414,-0.562713,-0.942337,2.0,-1.226491,2,1,1,-1.717891,0,1,1
5254,-0.500801,0.276889,10.0,-1.226491,2,0,0,1.205398,0,0,0
7154,0.066725,0.464462,6.0,-0.003919,1,1,0,-0.372328,0,0,1
2834,0.448515,-0.192044,10.0,1.072342,1,0,1,-1.166758,0,0,1
5592,-1.047689,-0.942337,3.0,-1.226491,2,1,0,0.791544,0,0,1


Как было замечено выше - данные не сбалансированы, они примерно соотносятся как 4:1. Попробуем построить модель по данным с дисбалансом и проверим качество модели. **Начнем с Обучающего дерева**

In [20]:
model = DecisionTreeClassifier(random_state=123, max_depth = 3)
model.fit(features_train_new, target_train_new)
predicted_valid_new = model.predict(features_valid_new)

print('Accuracy:', accuracy_score(target_valid_new, predicted_valid_new))
print('F1:', f1_score(target_valid_new, predicted_valid_new))

#Сравним модель со случайной
probabilities_valid_new = model.predict_proba(features_valid_new)
probabilities_one_valid_new = probabilities_valid_new[:, 1]

print('ROC_AUC:', roc_auc_score(target_valid_new, probabilities_one_valid_new))

Accuracy: 0.8416712479384277
F1: 0.4330708661417323
ROC_AUC: 0.784687471908429


**Случайный лес**

In [21]:
model = RandomForestClassifier(random_state=123)
model.fit(features_train_new, target_train_new)
predicted_valid_new = model.predict(features_valid_new)

print('Accuracy:', accuracy_score(target_valid_new, predicted_valid_new))
print('F1:', f1_score(target_valid_new, predicted_valid_new))

#Сравним модель со случайной
probabilities_valid_new = model.predict_proba(features_valid_new)
probabilities_one_valid_new = probabilities_valid_new[:, 1]

print('ROC_AUC:', roc_auc_score(target_valid_new, probabilities_one_valid_new))

Accuracy: 0.8532160527762507
F1: 0.5420240137221269
ROC_AUC: 0.8184011026878015


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


**Логистическая регрессия**

In [22]:
model = LogisticRegression(random_state=123, solver='liblinear')
model.fit(features_train_new, target_train_new)
predicted_valid_new = model.predict(features_valid_new)

print('Accuracy:', accuracy_score(target_valid_new, predicted_valid_new))
print('F1:', f1_score(target_valid_new, predicted_valid_new))

#Сравним модель со случайной
probabilities_valid_new = model.predict_proba(features_valid_new)
probabilities_one_valid_new = probabilities_valid_new[:, 1]

print('ROC_AUC:', roc_auc_score(target_valid_new, probabilities_one_valid_new))

Accuracy: 0.8158328752061572
F1: 0.31492842535787324
ROC_AUC: 0.7572494980972644


**Как можно заметить для всех трех моделей точность высока, также как и значение AUC_ROC - площадь под кривой, но данные модели плохо предсказывают целевой признак со значением класс 1, только случайный лес показывает значение превышающее 0.54, однако этого не хватает до 0.59, поэтому необходимо бороться с дисбалансом классов.**

**Однако можно заметить, что для второго варианта, когда удаляются пропущенные значения качество моделей немного выше, чем в случае с заменой пропущенных значений.**

## Борьба с дисбалансом

С дисбалансом признаков борятся многими методами, в данном проекте рассмотрим 3 из них: метод взвешивания классов, метод downsampling, метод upsampling. Первым рассмотрим метод взвешивание классов для каждой модели: обучающее дерево, случайный лес и логистическая регрессия. **Применим данный метод к первому датасету, в которойй производилась замена пропущенных значений.**

**Взвешивание классов**

**Обучающее дерево**

In [23]:
# Проверяем качество модели на валидационной выборке
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=123, max_depth=depth, class_weight = "balanced")
    model.fit(features_train_first, target_train_first)
    predictions_valid_first = model.predict(features_valid_first) 
    print("max_depth =", depth, ": ", end='')
    print(f1_score(target_valid_first, predictions_valid_first))
    probabilities_valid_first = model.predict_proba(features_valid_first)
    probabilities_one_valid_first = probabilities_valid_first[:, 1]
    print('ROC_AUC:', roc_auc_score(target_valid_first, probabilities_one_valid_first))

max_depth = 1 : 0.4749754661432777
ROC_AUC: 0.6884107538425243
max_depth = 2 : 0.4894837476099427
ROC_AUC: 0.7256959453789164
max_depth = 3 : 0.4894837476099427
ROC_AUC: 0.7797248361573333
max_depth = 4 : 0.5426944971537002
ROC_AUC: 0.8152421577988158
max_depth = 5 : 0.5303292894280762
ROC_AUC: 0.8208489892344402


Максимальное качество 0.54 достигается при количестве деревьев = 4, но этого не достаточно по условиям, однако при лучшем балансе классов, значение метрики увеличилось с 0,51 до 0.54, также как и показатель ROC_AUC.

**Случайный лес**

In [24]:
# Проверяем качество модели на валидационной выборке
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(1, 11):
    for depth in range(1, 6):
        model = RandomForestClassifier(random_state=123, n_estimators=est, max_depth = depth, class_weight = "balanced")
        model.fit(features_train_first, target_train_first)
        predicted_valid_first = model.predict(features_valid_first)
        result = f1_score(target_valid_first, predicted_valid_first)
        probabilities_valid_first = model.predict_proba(features_valid_first)
        probabilities_one_valid_first = probabilities_valid_first[:, 1]
        auc_roc = roc_auc_score(target_valid_first, probabilities_one_valid_first)
        if result > best_result:
            best_model = model 
            best_result = result
            best_est = est
            best_depth = depth
            best_auc_roc = auc_roc
            
print("F1-мера наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", best_depth, "Максимальный AUC_ROC:", best_auc_roc)

F1-мера наилучшей модели на валидационной выборке: 0.5643153526970954 Количество деревьев: 7 Максимальная глубина: 4 Максимальный AUC_ROC: 0.8259982170877728


Для случайного леса показатели метрики увеличилсь и стали 0.56, также они увеличились по сравнению с несбалансированным датасетом

**Логистическая регрессия**

In [25]:
model = LogisticRegression(random_state=123, solver='liblinear', class_weight = "balanced")
model.fit(features_train_first, target_train_first)
predicted_valid_first = model.predict(features_valid_first)
print('F1:', f1_score(target_valid_first, predicted_valid_first))
probabilities_valid_first = model.predict_proba(features_valid_first)
probabilities_one_valid_first = probabilities_valid_first[:, 1]
print('ROC_AUC:', roc_auc_score(target_valid_first, probabilities_one_valid_first))

F1: 0.5036496350364964
ROC_AUC: 0.7792909836260256


Как можно заметить балансирование классов значительно влияет на показатели метрики для логистической регрессии, он вырос с 0.3 до 0.5, однако этого еще не достаточно по условию задачи.

**Таким образом, можно заметить, что взвешивание классов увеличивает показатели качества по сравнению с несбалансированными данными, однако этих показателей еще не достаточно для выполнения условий задачи, поэтому перейдем к следующему способу борьбы с дисбалансом: upsampling.**

**UPsampling**

- Разделим обучающую выборку на отрицательные и положительные объекты;
- Скопируем несколько раз положительные объекты;
- С учётом полученных данных создадим новую обучающую выборку;
- Перемешаем данные: идущие друг за другом одинаковые вопросы не помогут обучению.

In [26]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=123)
    
    return features_upsampled, target_upsampled

# В качестве повтора поставим значение = 4, так как количество ответов класса 1 в 4 раза меньше, чем ответов класса 0
features_upsampled_first, target_upsampled_first = upsample(features_train_first, target_train_first, 4)


In [27]:
print(target_upsampled_first.value_counts())
print(target_train_first.value_counts())

1    4884
0    4779
Name: exited, dtype: int64
0    4779
1    1221
Name: exited, dtype: int64


Как можно заметить увеличение выборки прошло успешно, данные сбалансированы, можно приступить к проверке качества моделей

**Обучающее дерево**

In [28]:
# Проверяем качество модели на валидационной выборке
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=123, max_depth=depth)
    model.fit(features_upsampled_first, target_upsampled_first)
    predictions_valid_first = model.predict(features_valid_first) 
    print("max_depth =", depth, ": ", end='')
    print(f1_score(target_valid_first, predictions_valid_first))
    probabilities_valid_first = model.predict_proba(features_valid_first)
    probabilities_one_valid_first = probabilities_valid_first[:, 1]
    print('ROC_AUC:', roc_auc_score(target_valid_first, probabilities_one_valid_first))

max_depth = 1 : 0.4749754661432777
ROC_AUC: 0.6884107538425243
max_depth = 2 : 0.4894837476099427
ROC_AUC: 0.7256959453789164
max_depth = 3 : 0.4894837476099427
ROC_AUC: 0.7797248361573333
max_depth = 4 : 0.5426944971537002
ROC_AUC: 0.8152421577988158
max_depth = 5 : 0.5303292894280762
ROC_AUC: 0.8208489892344402


**Не совсем понимаю, почему для данной модели показатели метрик не изменились по сравнению со взвешиванием, хотя для случайного леса (ниже), поменялось.**

**Случайный лес**

In [29]:
# Проверяем качество модели на валидационной выборке
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(1, 100):
    for depth in range(1, 6):
        model = RandomForestClassifier(random_state=123, n_estimators=est, max_depth = depth)
        model.fit(features_upsampled_first, target_upsampled_first)
        predicted_valid_first = model.predict(features_valid_first)
        result = f1_score(target_valid_first, predicted_valid_first)
        probabilities_valid_first = model.predict_proba(features_valid_first)
        probabilities_one_valid_first = probabilities_valid_first[:, 1]
        auc_roc = roc_auc_score(target_valid_first, probabilities_one_valid_first)
        if result > best_result:
            best_model = model 
            best_result = result
            best_est = est
            best_depth = depth
            best_auc_roc = auc_roc
            
print("F1-мера наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", best_depth, "Максимальный AUC_ROC:", best_auc_roc)

F1-мера наилучшей модели на валидационной выборке: 0.5828092243186582 Количество деревьев: 28 Максимальная глубина: 5 Максимальный AUC_ROC: 0.8435280761173681


Для случайного леса показатели метрики увеличилсь и стали 0.582, по сравнению со взвешиванием, также они увеличились по сравнению с несбалансированным датасетом

**Логистическая регрессия**

In [30]:
model = LogisticRegression(random_state=123, solver='liblinear')
model.fit(features_upsampled_first, target_upsampled_first)
predicted_valid_first = model.predict(features_valid_first)
print('F1:', f1_score(target_valid_first, predicted_valid_first))
probabilities_valid_first = model.predict_proba(features_valid_first)
probabilities_one_valid_first = probabilities_valid_first[:, 1]
print('ROC_AUC:', roc_auc_score(target_valid_first, probabilities_one_valid_first))

F1: 0.5
ROC_AUC: 0.7793353189211956


Как можно заметить upsampling не изменил показатели качества для логистической регрессии.



Таким образом, можно заметить, что upsampling увеличивает показатели качества по сравнению с несбалансированными данными и по сравнению со взвешиванием классов для модели случайный лес, а для моделей обучающее дерево и логистическая регрессия показатели качества меняются незначительно, однако показателей случайного леса еще не достаточно для выполнения условий задачи, поэтому перейдем к следующему способу борьбы с дисбалансом: downsampling.

**DOWNsampling**

- Разделим обучающую выборку на отрицательные и положительные объекты;
- Случайным образом отбросим часть из отрицательных объектов;
- С учётом полученных данных создадим новую обучающую выборку;
- Перемешаем данные. Положительные не должны идти следом за отрицательными: алгоритмам будет сложнее обучаться.

In [31]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=123)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=123)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=123)
    
    return features_downsampled, target_downsampled

#Так как данные отличаются в соотношении 1:4, то уменьшим больший класс и поставим значение fraction = 0.25
features_downsampled_first, target_downsampled_first = downsample(features_train_first, target_train_first, 0.25)

In [32]:
print(target_downsampled_first.value_counts())
print(target_train_first.value_counts())

1    1221
0    1195
Name: exited, dtype: int64
0    4779
1    1221
Name: exited, dtype: int64


Как можно заметить уменьшение выборки прошло успешно, данные сбалансированы, можно приступить к проверке качества моделей

**Обучающее дерево**

In [33]:
# Проверяем качество модели на валидационной выборке
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=123, max_depth=depth)
    model.fit(features_downsampled_first, target_downsampled_first)
    predictions_valid_first = model.predict(features_valid_first) 
    print("max_depth =", depth, ": ", end='')
    print(f1_score(target_valid_first, predictions_valid_first))
    probabilities_valid_first = model.predict_proba(features_valid_first)
    probabilities_one_valid_first = probabilities_valid_first[:, 1]
    print('ROC_AUC:', roc_auc_score(target_valid_first, probabilities_one_valid_first))

max_depth = 1 : 0.4749754661432777
ROC_AUC: 0.6884107538425243
max_depth = 2 : 0.4894837476099427
ROC_AUC: 0.7200020267563506
max_depth = 3 : 0.5417721518987342
ROC_AUC: 0.7754710229260979
max_depth = 4 : 0.48874061718098416
ROC_AUC: 0.8037355652987644
max_depth = 5 : 0.535059331175836
ROC_AUC: 0.8055960642925116


Как можно заметить метрики качества для обучающего дерева увеличились до 0.542

**Случайный лес**

In [34]:
# Проверяем качество модели на валидационной выборке
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(1, 100):
    for depth in range(1, 6):
        model = RandomForestClassifier(random_state=123, n_estimators=est, max_depth = depth)
        model.fit(features_downsampled_first, target_downsampled_first)
        predicted_valid_first = model.predict(features_valid_first)
        result = f1_score(target_valid_first, predicted_valid_first)
        probabilities_valid_first = model.predict_proba(features_valid_first)
        probabilities_one_valid_first = probabilities_valid_first[:, 1]
        auc_roc = roc_auc_score(target_valid_first, probabilities_one_valid_first)
        if result > best_result:
            best_model = model 
            best_result = result
            best_est = est
            best_depth = depth
            best_auc_roc = auc_roc
            
print("F1-мера наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", best_depth, "Максимальный AUC_ROC:", best_auc_roc)

F1-мера наилучшей модели на валидационной выборке: 0.5829244357212953 Количество деревьев: 5 Максимальная глубина: 5 Максимальный AUC_ROC: 0.8366236455963176


Для случайного леса показатели метрики увеличилсь и стали 0.58, по сравнению со взвешиванием и upsampling, также они увеличились по сравнению с несбалансированным датасетом

**Логистическая регрессия**

In [35]:
model = LogisticRegression(random_state=123, solver='liblinear')
model.fit(features_downsampled_first, target_downsampled_first)
predicted_valid_first = model.predict(features_valid_first)
print('F1:', f1_score(target_valid_first, predicted_valid_first))
probabilities_valid_first = model.predict_proba(features_valid_first)
probabilities_one_valid_first = probabilities_valid_first[:, 1]
print('ROC_AUC:', roc_auc_score(target_valid_first, probabilities_one_valid_first))

F1: 0.5114155251141552
ROC_AUC: 0.7790946416045577


Как можно заметить downsampling изменил показатели качества для логистической регрессии c 0.5 до 0.51.

Таким образом, можно заметить, что downsampling увеличивает показатели качества по сравнению с несбалансированными данными и по сравнению со взвешиванием классов и upsampling для всех моделей, однако показателей еще не достаточно для выполнения условий задачи, поэтому перейдем к другому датасету

# **Датасет с удаленными пропущенными значениями**

**Метод downsampling**

In [36]:
features_downsampled_new, target_downsampled_new = downsample(features_train_new, target_train_new, 0.25)

In [37]:
print(target_downsampled_new.value_counts())
print(target_train_new.value_counts())

1    1101
0    1088
Name: exited, dtype: int64
0    4353
1    1101
Name: exited, dtype: int64


Как можно заметить уменьшение произведено корректно

In [38]:
# Проверяем качество модели на валидационной выборке
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(1, 100):
    for depth in range(1, 6):
        model = RandomForestClassifier(random_state=123, n_estimators=est, max_depth = depth)
        model.fit(features_downsampled_new, target_downsampled_new)
        predicted_valid_new = model.predict(features_valid_new)
        result = f1_score(target_valid_new, predicted_valid_new)
        probabilities_valid_new = model.predict_proba(features_valid_new)
        probabilities_one_valid_new = probabilities_valid_new[:, 1]
        auc_roc = roc_auc_score(target_valid_new, probabilities_one_valid_new)
        if result > best_result:
            best_model = model 
            best_result = result
            best_est = est
            best_depth = depth
            best_auc_roc = auc_roc
            
print("F1-мера наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", best_depth, "Максимальный AUC_ROC:", best_auc_roc)

F1-мера наилучшей модели на валидационной выборке: 0.5859030837004404 Количество деревьев: 14 Максимальная глубина: 5 Максимальный AUC_ROC: 0.8385024945315075


Получили качество близкое к нужному, но еще недостаточное, проверим метод upsampling для случайного леса, так как именно для этой модели получаются лучшие значения метрики качества.

**метод upsampling**

In [39]:
features_upsampled_new, target_upsampled_new = upsample(features_train_new, target_train_new, 4)

In [40]:
print(target_upsampled_new.value_counts())
print(target_train_new.value_counts())

1    4404
0    4353
Name: exited, dtype: int64
0    4353
1    1101
Name: exited, dtype: int64


In [41]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(1, 100):
    for depth in range(1, 6):
        model = RandomForestClassifier(random_state=123, n_estimators=est, max_depth = depth)
        model.fit(features_upsampled_new, target_upsampled_new)
        predicted_valid_new = model.predict(features_valid_new)
        result = f1_score(target_valid_new, predicted_valid_new)
        probabilities_valid_new = model.predict_proba(features_valid_new)
        probabilities_one_valid_new = probabilities_valid_new[:, 1]
        auc_roc = roc_auc_score(target_valid_new, probabilities_one_valid_new)
        if result > best_result:
            best_model = model 
            best_result = result
            best_est = est
            best_depth = depth
            best_auc_roc = auc_roc
            
print("F1-мера наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", best_depth, "Максимальный AUC_ROC:", best_auc_roc)

F1-мера наилучшей модели на валидационной выборке: 0.5922651933701657 Количество деревьев: 38 Максимальная глубина: 5 Максимальный AUC_ROC: 0.8401608336080064


Необходимое качство метрики для данного метода удовлетворяет условию задачи, но на всякий случай проверим также взвешивание классов

**Взвешивание классов**

In [42]:
# Проверяем качество модели на валидационной выборке
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(1, 11):
    for depth in range(1, 6):
        model = RandomForestClassifier(random_state=123, n_estimators=est, max_depth = depth, class_weight = "balanced")
        model.fit(features_train_new, target_train_new)
        predicted_valid_new = model.predict(features_valid_new)
        result = f1_score(target_valid_new, predicted_valid_new)
        probabilities_valid_new = model.predict_proba(features_valid_new)
        probabilities_one_valid_new = probabilities_valid_new[:, 1]
        auc_roc = roc_auc_score(target_valid_new, probabilities_one_valid_new)
        if result > best_result:
            best_model = model 
            best_result = result
            best_est = est
            best_depth = depth
            best_auc_roc = auc_roc
            
print("F1-мера наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", best_depth, "Максимальный AUC_ROC:", best_auc_roc)

F1-мера наилучшей модели на валидационной выборке: 0.5856980703745743 Количество деревьев: 6 Максимальная глубина: 4 Максимальный AUC_ROC: 0.8337737467413777


Данная метрика близка к уловию задачи, но не подходит.

**Таким образом, с несбалансированными данными была произведена работа тремя методами:upsampling, downsampling и взвешивание классов для обеих выборок. Лучше всего себя показали методы:**
- upsampling для модели случайный лес для датасета, в котором были удалены пропущенные значения - метрика = 0.592;
- upsampling для модели случайный лес для датасета, в котором были заменены пропущенные значения - метрика = 0.582;
- downsampling для модели случайный лес для датасета, в котором были заменены пропущенные значения - метрика = 0.583;
- downsampling для модели случайный лес для датасета, в котором были удалены пропущенные значения - метрика = 0.586;
- взвешивание для модели случайный лес для датасета, в котором были удалены пропущенные значения - метрика = 0.5857.

**Удовлетворяет необходимому условию задачи только первый вариант, однако для проверки, протестируем на тестовой выборке все 5 вариантов.**

## Тестирование модели

**Для начала проверим второй датасет с удаленными пропущенными значениями**

**Модель со значением метрики на валидационной выборке = 0.592 - методо upsampling**

In [43]:
model_best = RandomForestClassifier(random_state=123, n_estimators=38, max_depth = 5)
model_best.fit(features_upsampled_new, target_upsampled_new)
predicted_test_new = model_best.predict(features_test_new)
result = f1_score(target_test_new, predicted_test_new)
print("F1:", result)
probabilities_test_new = model_best.predict_proba(features_test_new)
probabilities_one_test_new = probabilities_test_new[:, 1]
auc_roc = roc_auc_score(target_test_new, probabilities_one_test_new)
print("AUC_ROC:", auc_roc)

F1: 0.5890985324947589
AUC_ROC: 0.8481371385069919


Как можно заметить тестовая выборка не прошла пороговое значение 0.59, проверим метрику для метода downsampling для того же датасета

**Модель со значением метрики на валидационной выборке = 0.586 - метод dowmsampling**

In [44]:
model_best = RandomForestClassifier(random_state=123, n_estimators=14, max_depth = 5)
model_best.fit(features_downsampled_new, target_downsampled_new)
predicted_test_new = model_best.predict(features_test_new)
result = f1_score(target_test_new, predicted_test_new)
print("F1:", result)
probabilities_test_new = model_best.predict_proba(features_test_new)
probabilities_one_test_new = probabilities_test_new[:, 1]
auc_roc = roc_auc_score(target_test_new, probabilities_one_test_new)
print("AUC_ROC:", auc_roc)

F1: 0.5813221406086044
AUC_ROC: 0.8459974080350912


Проверка на тестовой выборке также не прошла пороговое значение, проверим метод взвешивание классов

**Модель со значением метрики на валидационной выборке = 0.586 - метод взвешивание классов**

In [45]:
model_best = RandomForestClassifier(random_state=123, n_estimators=6, max_depth = 4, class_weight = 'balanced')
model_best.fit(features_train_new, target_train_new)
predicted_test_new = model.predict(features_test_new)
result = f1_score(target_test_new, predicted_test_new)
print("F1:", result)
probabilities_test_new = model.predict_proba(features_test_new)
probabilities_one_test_new = probabilities_test_new[:, 1]
auc_roc = roc_auc_score(target_test_new, probabilities_one_test_new)
print("AUC_ROC:", auc_roc)

F1: 0.576307363927428
AUC_ROC: 0.8353884775378146


Проверка на тестовой выборке также не прошла пороговое значение, проверим другой датасет, с заменой пропущенных значений и метод upsampling

**Модель со значением метрики на валидационной выборке = 0.582 - метод upsampling**

In [46]:
model_best = RandomForestClassifier(random_state=123, n_estimators=28, max_depth = 5)
model_best.fit(features_upsampled_first, target_upsampled_first)
predicted_test_first = model_best.predict(features_test_first)
result = f1_score(target_test_first, predicted_test_first)
print("F1:", result)
probabilities_test_first = model_best.predict_proba(features_test_first)
probabilities_one_test_first = probabilities_test_first[:, 1]
auc_roc = roc_auc_score(target_test_first, probabilities_one_test_first)
print("AUC_ROC:", auc_roc)

F1: 0.6109452736318409
AUC_ROC: 0.8567281443804333


Проверка на тестовой выборке прошла пороговое значение в 0.59, но на всякий случай проверим метод downsampling

**Модель со значением метрики на валидационной выборке = 0.583 - метод downsampling**

In [47]:
model_best = RandomForestClassifier(random_state=123, n_estimators=5, max_depth = 5)
model_best.fit(features_downsampled_first, target_downsampled_first)
predicted_test_first = model_best.predict(features_test_first)
result = f1_score(target_test_first, predicted_test_first)
print("F1:", result)
probabilities_test_first = model_best.predict_proba(features_test_first)
probabilities_one_test_first = probabilities_test_first[:, 1]
auc_roc = roc_auc_score(target_test_first, probabilities_one_test_first)
print("AUC_ROC:", auc_roc)

F1: 0.5741444866920152
AUC_ROC: 0.8344134282557628


Проверка на тестовой выборке не прошла пороговое значение в 0.59.

**Таким образом, несмотря на то, что при проверке на валидационной выборке лучше всего себя показала модель случайный лес на датасете с удаленными пропущенными значениями и метод upsampling, на тестовых выборках лучше всего себя показал датасет с заменой пропущенных значений с методом upsampling и моделью случайный лес. Его показатели качества: F1-мера = 0.61, AUC_ROC = 0.857, что удовлетворяет условиям задачи.**

**Для предсказания ухода клиента из банка можно использовать модель случайный лес, в которой данные сбалансированы методом upsampling и пропущенные значения заменены на медиану**

# Датасет с удалением признака "Tenure"

In [50]:
df_drop = df.drop(['tenure'], axis = 1)
print(df_drop.shape)
df_drop

(10000, 10)


,credit_score,geography,gender,age,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,619,France,Female,42,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,75075.31,2,1,0,92888.52,1


In [51]:
df_drop_ohe = pd.get_dummies(df_drop, drop_first = True)

In [52]:
train_drop = df_drop_ohe.sample(frac = 0.8, random_state = 123).copy()
print(train_drop.shape)

test_drop = df_drop_ohe[~df_drop_ohe.index.isin(train_drop.index)].copy()
print(test_drop.shape)

(8000, 11)
(2000, 11)


In [53]:
df_train_drop, df_valid_drop = train_test_split(train_drop, test_size=0.25, random_state=123)
print(df_train_drop.shape)
print(df_valid_drop.shape)

(6000, 11)
(2000, 11)


In [54]:
features_train_drop = df_train_drop.drop(['exited'], axis=1)
target_train_drop = df_train_drop['exited']

features_valid_drop = df_valid_drop.drop(['exited'], axis=1)
target_valid_drop = df_valid_drop['exited']

features_test_drop = test_drop.drop(['exited'], axis=1)
target_test_drop = test_drop['exited']

print(features_train_drop.shape)
print(target_train_drop.shape)

print(features_valid_drop.shape)
print(target_valid_drop.shape)

print(features_test_drop.shape)
print(target_test_drop.shape)

(6000, 10)
(6000,)
(2000, 10)
(2000,)
(2000, 10)
(2000,)


In [57]:
# Выделяем те признаки, которые необходимо масштабировать
numeric = ['credit_score', 'age', 'balance', 'estimated_salary']

scaler = StandardScaler()
scaler.fit(features_train_drop[numeric])
features_train_drop[numeric] =  scaler.transform(features_train_drop[numeric])
features_valid_drop[numeric] =  scaler.transform(features_valid_drop[numeric])
features_test_drop[numeric] =  scaler.transform(features_test_drop[numeric])
features_train_drop.head()

,credit_score,age,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,geography_Germany,geography_Spain,gender_Male
2256,0.479630,-0.568592,1.393707,1,1,1,-1.609849,0,0,1
9202,-1.257624,-0.759496,0.715344,1,1,0,-1.638045,0,0,1
8544,0.407245,0.672285,-0.393582,2,1,1,-0.459033,0,0,0
8818,0.283155,-0.186784,-1.230201,2,1,1,-1.065188,0,0,0
8017,-1.971139,1.054093,0.529178,1,1,0,0.984366,1,0,1


**Взвешивание классов**

In [58]:
# Проверяем качество модели на валидационной выборке
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(1, 11):
    for depth in range(1, 6):
        model = RandomForestClassifier(random_state=123, n_estimators=est, max_depth = depth, class_weight = "balanced")
        model.fit(features_train_drop, target_train_drop)
        predicted_valid_drop = model.predict(features_valid_drop)
        result = f1_score(target_valid_drop, predicted_valid_drop)
        probabilities_valid_drop = model.predict_proba(features_valid_drop)
        probabilities_one_valid_drop = probabilities_valid_drop[:, 1]
        auc_roc = roc_auc_score(target_valid_drop, probabilities_one_valid_drop)
        if result > best_result:
            best_model = model 
            best_result = result
            best_est = est
            best_depth = depth
            best_auc_roc = auc_roc
            
print("F1-мера наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", best_depth, "Максимальный AUC_ROC:", best_auc_roc)

F1-мера наилучшей модели на валидационной выборке: 0.5773195876288659 Количество деревьев: 10 Максимальная глубина: 5 Максимальный AUC_ROC: 0.8440846424120934


**Метод upsampling**

In [59]:
features_upsampled_drop, target_upsampled_drop = upsample(features_train_drop, target_train_drop, 4)
print(target_upsampled_drop.value_counts())
print(target_train_drop.value_counts())

1    4884
0    4779
Name: exited, dtype: int64
0    4779
1    1221
Name: exited, dtype: int64


In [60]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(1, 100):
    for depth in range(1, 6):
        model = RandomForestClassifier(random_state=123, n_estimators=est, max_depth = depth)
        model.fit(features_upsampled_drop, target_upsampled_drop)
        predicted_valid_drop = model.predict(features_valid_drop)
        result = f1_score(target_valid_drop, predicted_valid_drop)
        probabilities_valid_drop = model.predict_proba(features_valid_drop)
        probabilities_one_valid_drop = probabilities_valid_drop[:, 1]
        auc_roc = roc_auc_score(target_valid_drop, probabilities_one_valid_drop)
        if result > best_result:
            best_model = model 
            best_result = result
            best_est = est
            best_depth = depth
            best_auc_roc = auc_roc
            
print("F1-мера наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", best_depth, "Максимальный AUC_ROC:", best_auc_roc)

F1-мера наилучшей модели на валидационной выборке: 0.5798575788402849 Количество деревьев: 99 Максимальная глубина: 5 Максимальный AUC_ROC: 0.8489401489349238


**Метод downsampling**

In [61]:
features_downsampled_drop, target_downsampled_drop = downsample(features_train_drop, target_train_drop, 0.25)
print(target_downsampled_drop.value_counts())
print(target_train_drop.value_counts())

1    1221
0    1195
Name: exited, dtype: int64
0    4779
1    1221
Name: exited, dtype: int64


In [62]:
# Проверяем качество модели на валидационной выборке
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(1, 100):
    for depth in range(1, 6):
        model = RandomForestClassifier(random_state=123, n_estimators=est, max_depth = depth)
        model.fit(features_downsampled_drop, target_downsampled_drop)
        predicted_valid_drop = model.predict(features_valid_drop)
        result = f1_score(target_valid_drop, predicted_valid_drop)
        probabilities_valid_drop = model.predict_proba(features_valid_drop)
        probabilities_one_valid_drop = probabilities_valid_drop[:, 1]
        auc_roc = roc_auc_score(target_valid_drop, probabilities_one_valid_drop)
        if result > best_result:
            best_model = model 
            best_result = result
            best_est = est
            best_depth = depth
            best_auc_roc = auc_roc
            
print("F1-мера наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", best_depth, "Максимальный AUC_ROC:", best_auc_roc)

F1-мера наилучшей модели на валидационной выборке: 0.5783836416747808 Количество деревьев: 24 Максимальная глубина: 5 Максимальный AUC_ROC: 0.8452294430695224


**Тестирование метод upsampling**

In [63]:
model_best = RandomForestClassifier(random_state=123, n_estimators=99, max_depth = 5)
model_best.fit(features_upsampled_drop, target_upsampled_drop)
predicted_test_drop = model_best.predict(features_test_drop)
result = f1_score(target_test_drop, predicted_test_drop)
print("F1:", result)
probabilities_test_drop = model_best.predict_proba(features_test_drop)
probabilities_one_test_drop = probabilities_test_drop[:, 1]
auc_roc = roc_auc_score(target_test_drop, probabilities_one_test_drop)
print("AUC_ROC:", auc_roc)

F1: 0.6068292682926829
AUC_ROC: 0.8618460403765116


**Тестирование метод downsampling**

In [64]:
model_best = RandomForestClassifier(random_state=123, n_estimators=24, max_depth = 5)
model_best.fit(features_downsampled_drop, target_downsampled_drop)
predicted_test_drop = model_best.predict(features_test_drop)
result = f1_score(target_test_drop, predicted_test_drop)
print("F1:", result)
probabilities_test_drop = model_best.predict_proba(features_test_drop)
probabilities_one_test_drop = probabilities_test_drop[:, 1]
auc_roc = roc_auc_score(target_test_drop, probabilities_one_test_drop)
print("AUC_ROC:", auc_roc)

F1: 0.5964912280701754
AUC_ROC: 0.856040811247978


**Тестирование метод взвешивание**

In [65]:
model_best = RandomForestClassifier(random_state=123, n_estimators=6, max_depth = 4, class_weight = 'balanced')
model_best.fit(features_train_drop, target_train_drop)
predicted_test_drop = model.predict(features_test_drop)
result = f1_score(target_test_drop, predicted_test_drop)
print("F1:", result)
probabilities_test_drop = model.predict_proba(features_test_drop)
probabilities_one_test_drop = probabilities_test_drop[:, 1]
auc_roc = roc_auc_score(target_test_drop, probabilities_one_test_drop)
print("AUC_ROC:", auc_roc)

F1: 0.5946969696969697
AUC_ROC: 0.8597630537079262


**Таким образом, можно сказать, что удаление признака "tenure" полностью, значительно повлияло на показатели качества модели для тестовой выборки, при всех методах достигается положительный результат качества, который удовлетворяет условию задачи, но лучшим показателем стала модель случайный лес, при методе борьбы с дисбалансом upsampling, с мерой F1 = 0.607**